## 步骤1 导入包

In [1]:
# 学术资源加速
import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import *

## 步骤2 获取数据集

In [3]:
# import pandas as pd


# data = pd.read_csv("/root/transformers/classdata/ChnSentiCorp_htl_all.csv")
# data.dropna()
# data


## 修改为
dataset = load_dataset("csv", data_files="/root/transformers/classdata/ChnSentiCorp_htl_all.csv", split="train")
dataset = dataset.filter(lambda x: x["review"] is not None)
dataset

Dataset({
    features: ['label', 'review'],
    num_rows: 7765
})

## 步骤3 创建数据集

In [5]:
# from torch.utils.data import Dataset

# import pandas as pd



# class MyDataset(Dataset):
#     def __init__(self) -> None:
#         super().__init__()
#         self.data = pd.read_csv("/root/transformers/classdata/ChnSentiCorp_htl_all.csv")
#         self.data = self.data.dropna()
#     def __getitem__(self,index):
#         return self.data.iloc[index]["review"], self.data.iloc[index]["label"]
#     def __len__(self):
#         return len(self.data)
    

    

# dataset = MyDataset()
# for i in range(5):
#     print(dataset[i])

('距离川沙公路较近,但是公交指示不对,如果是"蔡陆线"的话,会非常麻烦.建议用别的路线.房间较为简单.', 1)
('商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!', 1)
('早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。', 1)
('宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小，但加上低价位因素，还是无超所值的；环境不错，就在小胡同内，安静整洁，暖气好足-_-||。。。呵还有一大优势就是从宾馆出发，步行不到十分钟就可以到梅兰芳故居等等，京味小胡同，北海距离好近呢。总之，不错。推荐给节约消费的自助游朋友~比较划算，附近特色小吃很多~', 1)
('CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风', 1)


## 步骤4 划分数据集

In [4]:
# from torch.utils.data import  random_split



# trainset, validset = random_split(dataset,lengths=[0.9,0.1])
# len(trainset),len(validset)

# 修改为：

datasets = dataset.train_test_split(test_size=0.1)
datasets

DatasetDict({
    train: Dataset({
        features: ['label', 'review'],
        num_rows: 6988
    })
    test: Dataset({
        features: ['label', 'review'],
        num_rows: 777
    })
})

## 步骤5 创建DataLoader

In [5]:
# import torch
# from torch.utils.data import DataLoader


# tokenizer = AutoTokenizer.from_pretrained("/root/transformers/classmodel/rbt3")

# def collate_func(batch):
#     texts,labels=[],[]
#     for item in batch:
#         texts.append(item[0])
#         labels.append(item[1])
#         ## return_tensors="pt" 返回的是pytorch tensor类型。
#         ## 吃葡萄不吐葡萄皮
#         ## 不吃葡萄到吐葡萄皮
#     inputs = tokenizer(texts,max_length=128,padding="max_length",truncation=True, return_tensors="pt")
#     inputs["labels"] = torch.tensor(labels)
#     return inputs
# ## dataloader中设置shuffle值为True，表示每次加载的数据都是随机的，将输入数据的顺序打乱。shuffle值为False，
# ## 表示输入数据顺序固定。


# trainloader = DataLoader(trainset,batch_size=32,shuffle=True,collate_fn=collate_func)
# validloader = DataLoader(validset,batch_size=64,shuffle=False,collate_fn=collate_func)

# next(enumerate(validloader))[1]

## 修改为
import torch
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding

tokenizer = AutoTokenizer.from_pretrained("/root/transformers/classmodel/rbt3")

def process_function(examples):
    tokenized_examples = tokenizer(examples["review"], max_length=128, truncation=True)
    tokenized_examples["labels"] = examples["label"]
    return tokenized_examples

tokenized_datasets = datasets.map(process_function, batched=True, remove_columns=datasets["train"].column_names)
## list(list,list) pt

trainset, validset = tokenized_datasets["train"], tokenized_datasets["test"]
# DataCollatorWithPadding进行批量补齐操作（同时作用于input_ids、attention_mask和token_type_ids）
trainloader = DataLoader(trainset, batch_size=32, shuffle=True, collate_fn=DataCollatorWithPadding(tokenizer))
validloader = DataLoader(validset, batch_size=64, shuffle=False, collate_fn=DataCollatorWithPadding(tokenizer))
next(enumerate(validloader))[1]

Map:   0%|          | 0/6988 [00:00<?, ? examples/s]

Map:   0%|          | 0/777 [00:00<?, ? examples/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'input_ids': tensor([[ 101, 6983, 2421,  ...,    0,    0,    0],
        [ 101, 3403, 1114,  ...,    0,    0,    0],
        [ 101, 4384, 1862,  ..., 1736, 1139,  102],
        ...,
        [ 101, 1348, 6206,  ...,    0,    0,    0],
        [ 101, 2791, 7313,  ...,    0,    0,    0],
        [ 101, 1282,  671,  ..., 7667, 1762,  102]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1]]), 'labels': tensor([1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1,
        1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1,
        1, 0, 0

## 步骤6 创建模型及其优化器

In [6]:
from torch.optim import Adam

from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("/root/transformers/classmodel/rbt3")

if torch.cuda.is_available():
    model = model.cuda()
"""
当我们训练一个机器学习模型时，我们需要选择一个优化算法来帮助我们找到模型参数的最佳值。这个优化算法就是优化器（optimizer）。

在这行代码中，我们选择了一种叫做Adam的优化算法作为我们的优化器。Adam算法是一种常用的优化算法，
它根据每个参数的梯度（即参数的变化率）和学习率（lr）来更新参数的值。

"model.parameters()"表示我们要优化的是模型的参数。模型的参数是模型中需要学习的权重和偏置等变量。

"lr=2e-5"表示学习率的值被设置为2e-5（即0.00002）。学习率是控制模型在每次迭代中更新参数的步长。较大的学习率可能导致模型无法收敛，
而较小的学习率可能需要更长的训练时间
"""
optimizer = Adam(model.parameters(), lr=2e-5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /root/transformers/classmodel/rbt3 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## 步骤7 训练与验证

In [10]:
import evaluate
clf_metrics = evaluate.combine(["accuracy", "f1"])

def evaluate():
    model.eval()
    with torch.inference_mode():
        for batch in validloader:
            if torch.cuda.is_available():
                batch = {k: v.cuda() for k, v in batch.items()}
            output = model(**batch)
            pred = torch.argmax(output.logits, dim=-1)
            clf_metrics.add_batch(predictions=pred.long(), references=batch["labels"].long())
    return clf_metrics.compute()


# def evaluate():
#     ## 将模型设置为评估模式
#     model.eval()
#     acc_num=0
#     #将训练模型转化为推理模型，模型将使用转换后的推理模式进行评估
#     with torch.inference_mode():
#         for batch in validloader:
#             ## 检查是否有可用的GPU，如果有，则将数据批次转移到GPU上进行加速
#             if torch.cuda.is_available():
#                 batch = {k: v.cuda() for k,v in batch.items()}
#             ##对数据批次进行前向传播，得到模型的输出
#             output = model(**batch)
#             ## 对模型输出进行预测，通过torch.argmax选择概率最高的类别。
#             pred = torch.argmax(output.logits,dim=-1)
#             ## 计算正确预测的数量，将预测值与标签进行比较，并使用.float()将比较结果转换为浮点数，使用.sum()进行求和操作
#             acc_num += (pred.long() == batch["labels"].long()).float().sum()
#     ## 返回正确预测数量与验证集样本数量的比值，这表示模型在验证集上的准确率
#     return acc_num / len(validset)




def train(epoch=3,log_sep=100):
    global_step = 0
    for ep in range(epoch):
        ## 开启训练模式
        model.train()
        for batch in trainloader:
            if torch.cuda.is_available():
                batch = {k: v.cuda() for k, v in batch.items()}
            ## 梯度归0
            optimizer.zero_grad()
            ## 对数据批次进行前向传播，得到模型的输出
            output=model(**batch)
            ## 计算损失函数梯度并进行反向传播
            output.loss.backward()
            ## 优化器更新
            optimizer.step()
            if(global_step % log_sep == 0):
                print(f"ep:{ep},global_step:{global_step},loss:{output.loss.item()}")
            global_step += 1
        ## 准确率
        clf = evaluate()
        ## 第几轮
        print(f"ep:{ep}, {clf}")




In [11]:
train()

ep:0,global_step:0,loss:0.6009122729301453
ep:0,global_step:100,loss:0.3029576241970062
ep:0,global_step:200,loss:0.40929195284843445
ep:0,acc:0.8893178701400757
ep:1,global_step:300,loss:0.12451878190040588
ep:1,global_step:400,loss:0.4621058702468872
ep:1,acc:0.8867439031600952
ep:2,global_step:500,loss:0.1324274092912674
ep:2,global_step:600,loss:0.41864141821861267
ep:2,acc:0.8906049132347107


## 步骤8 模型预测

In [15]:
#sen = "昨晚在酒店里我睡得很好"

# id2label = {0:"差评",1:"好评"}
# ## 将模型设置为评估模式
# model.eval

#  #将训练模型转化为推理模型，模型将使用转换后的推理模式进行评估
# with torch.inference_mode():
#     ## 分词&&
#     inputs = tokenizer(sen,return_tensors = "pt")
#     ## GPU加速
#     inputs = {k:v.cuda() for k,v in inputs.items()}
#     ## 进行预测
#     logits=model(**inputs).logits
#     ## 在logits的最后一个维度上找到最大值，并返回其所在的索引。这相当于选择模型认为最有可能的类别
#     pred = torch.argmax(logits, dim = -1)
    
#     print(f"输入：{sen} \n模型的预测结果：{id2label.get(pred.item())}")

输入：昨晚在酒店里我睡得很好 
模型的预测结果：好评


## 使用Pipeline优化

In [12]:
from transformers import pipeline

#model.config.id2label = id2label
sen = "昨晚在酒店里我睡得很好"

pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, device=0)

In [13]:
pipe(sen)

[{'label': 'LABEL_1', 'score': 0.9283226132392883}]